In [1]:
import numpy as np
import pandas as pd
import datetime
import urllib

In [2]:
from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict

## Read in our data. We've aggregated it by date already, so we don't need to worry about paging

In [3]:
query = ("https://data.lacity.org/resource/mgue-vbsx.json?"
    "$group=date"
    "&call_type_code=507P"
    "&$select=date_trunc_ymd(dispatch_date)%20AS%20date%2C%20count(*)"
    "&$order=date")

In [4]:
raw_data = pd.read_json(query)

In [6]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 2 columns):
count    364 non-null int64
date     364 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.8 KB


## Augment the data frame with the day of the week and the start of the week that it's in.

In [7]:
raw_data['day_of_week'] = [date.dayofweek for date in raw_data["date"]]
raw_data['week'] = [(date - datetime.timedelta(days=date.dayofweek)).strftime("%Y-%m-%d") for date in raw_data["date"]]

In [8]:
raw_data.head()

,count,date,day_of_week,week
0,285,2014-01-01,2,2013-12-30
1,16,2014-01-02,3,2013-12-30
2,33,2014-01-03,4,2013-12-30
3,134,2014-01-04,5,2013-12-30
4,101,2014-01-05,6,2013-12-30


## Pivot our data to get the matrix we need

In [9]:
data = raw_data.pivot(index='week', columns='day_of_week', values='count')
data = data.fillna(value=0)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53 entries, 2013-12-30 to 2014-12-29
Data columns (total 7 columns):
0    53 non-null float64
1    53 non-null float64
2    53 non-null float64
3    53 non-null float64
4    53 non-null float64
5    53 non-null float64
6    53 non-null float64
dtypes: float64(7)
memory usage: 3.3+ KB


In [11]:
data.head()

day_of_week,0,1,2,3,4,5,6
week,,,,,,,
2013-12-30,0.0,0.0,285.0,16.0,33.0,134.0,101.0
2014-01-06,11.0,17.0,12.0,10.0,55.0,178.0,146.0
2014-01-13,13.0,11.0,13.0,17.0,50.0,233.0,174.0
2014-01-20,28.0,8.0,13.0,14.0,47.0,262.0,170.0
2014-01-27,24.0,13.0,14.0,19.0,41.0,181.0,131.0


## Get our "weeks" and "days"

In [12]:
weeks = list(data.index)
days = ["Mon", "Tues", "Wed", "Thurs", "Fri", "Sat", "Sun"]

## Set up the data for plotting. We will need to have values for every pair of year/month names. Map the rate to a color.

In [13]:
max_count = raw_data["count"].max()
day_of_week = []
week = []
color = []
parties = []
for w in weeks:
    for idx, day in enumerate(days):
        day_of_week.append(day)
        week.append(w)
        count = data.loc[w][idx]
        parties.append(count)
        color.append("#%02x%02x%02x" % (255, 255 - (count / max_count) * 255.0, 255 - (count / max_count) * 255.0))

In [14]:
source = ColumnDataSource(
    data=dict(
        day_of_week=day_of_week,
        week=week,
        color=color,
        parties=parties,
    )
)

In [15]:
output_file('all-la-parties.html')

In [16]:
TOOLS = "hover"

In [17]:
p=figure(
    title='\"Party\" Disturbance Calls in LA', 
    x_range=weeks, 
    y_range=list(reversed(days)),
    tools=TOOLS)
p.plot_width=900
p.plot_height = 400
p.toolbar_location='left'

In [18]:
p.rect("week", "day_of_week", 1, 1, source=source, color=color, line_color=None)

C:\ProgramData\Anaconda2\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


GlyphRenderer(id='6aefaa32-689f-4e92-ab6e-a25d4a9b397a', ...)

In [19]:
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "10pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3

In [20]:
hover = p.select(dict(type=HoverTool))
hover.tooltips = OrderedDict([
    ('parties', '@parties'),
])

In [21]:
show(p) # show the plot